# Random Forest Regression

In [4]:
#Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

### We load the data.

In [5]:
#We load the data
data_path = "Data/GSM3130435_egfp_unmod_1_PREPROCESSED.csv.gz"
df = pd.read_csv(data_path, compression='gzip')

In [6]:
df.head()

,Unnamed: 0,utr,0,1,2,3,4,5,6,7,...,r9,r10,r11,r12,r13,rl,Length,Selection,one-hot encoding,scaled_rl
0,120605,CCACTCGATTAACATGTTAACAACATACTCGTCCGGCCGATCAGCG...,0.000137,0.000109,0.000057,0.000033,0.000015,0.000016,0.000009,0.000009,...,0.019283,0.033252,0.033252,0.027581,0.035877,3.039939,50,Selected Data,"[[0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0,...",-2.130922
1,11605,CAAATCATGTGCAGCCCTGGCGACCGTACTGCGGTACAAGAAAGTA...,0.000067,0.000070,0.000065,0.000048,0.000023,0.000020,0.000011,0.000011,...,0.024241,0.039457,0.039457,0.039840,0.038785,3.895109,50,Selected Data,"[[0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1,...",-1.600332
2,128224,GTTATACTAGAAGAAACTTGAGATTATGGAGCAGTCCGTCAAGGAC...,0.000088,0.000081,0.000059,0.000035,0.000017,0.000016,0.000009,0.000009,...,0.021591,0.028353,0.028353,0.028963,0.041985,3.334524,50,Selected Data,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 1], [1,...",-1.948147
3,239107,CTTAGACAAAAACAACGCGCTTTCCAGTATGCGGAGCCTTGACGGT...,0.000078,0.000071,0.000060,0.000038,0.000025,0.000016,0.000010,0.000009,...,0.026617,0.038302,0.038302,0.032788,0.031043,3.575082,50,Selected Data,"[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [1,...",-1.798893
4,59082,GTATCAAATCACGGCCAACCCGACGGAGTACCCCGCGTCGATGGTC...,0.000044,0.000045,0.000050,0.000051,0.000033,0.000026,0.000015,0.000012,...,0.033080,0.051449,0.051449,0.046052,0.036447,4.593712,50,Selected Data,"[[0, 0, 1, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0,...",-1.166885


In [7]:
#We select the highest quality data like in the paper (data with higest total reads)
df.sort_values('total_reads', inplace=True, ascending=False)
df.reset_index(inplace=True, drop=True)
df = df.iloc[:280000]

### We process the data.

In [8]:
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [9]:
df['words'] = df['utr'].apply(lambda x: getKmers(x))
df['words'].head()

0    [ccactc, cactcg, actcga, ctcgat, tcgatt, cgatt...
1    [caaatc, aaatca, aatcat, atcatg, tcatgt, catgt...
2    [gttata, ttatac, tatact, atacta, tactag, actag...
3    [cttaga, ttagac, tagaca, agacaa, gacaaa, acaaa...
4    [gtatca, tatcaa, atcaaa, tcaaat, caaatc, aaatc...
Name: words, dtype: object

In [10]:
df_text = list(df['words'])
for item in range(len(df_text)):
    df_text[item] = ' '.join(df_text[item])
y_data = df['rl'].values  

In [11]:
df_text[0]

'ccactc cactcg actcga ctcgat tcgatt cgatta gattaa attaac ttaaca taacat aacatg acatgt catgtt atgtta tgttaa gttaac ttaaca taacaa aacaac acaaca caacat aacata acatac catact atactc tactcg actcgt ctcgtc tcgtcc cgtccg gtccgg tccggc ccggcc cggccg ggccga gccgat ccgatc cgatca gatcag atcagc tcagcg cagcgg agcggc gcggct cggcta'

In [12]:
y_data

array([3.039939  , 3.89510901, 3.33452431, ..., 6.14800385, 6.5360733 ,
       4.7711601 ])

In [13]:
cv = CountVectorizer(ngram_range=(3,3)) #ngram_range=(4,4))
X = cv.fit_transform(df_text)

### We split our dataset in training set and testing set. 

In [14]:
# Splitting the human dataset into the training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y_data, 
                                                    test_size = 0.20, 
                                                    random_state=42)

### We transform our problem in a classification task.

In [18]:
#Random Forest Model
model = RandomForestRegressor(n_estimators = 30)

In [ ]:
%%time
rfr = model.fit(X_train, y_train)

In [ ]:
#We compute r square
print('The training r_squarred is: {}'.format(rfr.score(X_train, y_train)))

In [ ]:
#We compute r square
print('The testing r_squarred is: {}'.format(rfr.score(X_test, y_test)))

In [ ]:
model.save('./saved_models/Random_Forest_k_mer_counting.hdf5')

In [15]:
ypred = rfr.predict(X_test)

mse = mean_squared_error(y_test, ypred)
print("MSE: ", mse)
print("RMSE: ", mse*(1/2.0)) 


NameError: name 'mean_squared_error' is not defined